In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from warnings import filterwarnings
filterwarnings("ignore")
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

* Given that there are several datasets. We will do an EDA of each step by step beginning with


# Stock_list.csv

In [ ]:
list = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")
list.head()

* Primary observation shows several missing / NaN values. 


In [ ]:
list.shape

* There are 4417 samples/instances and 16 features/columns

In [ ]:
list.describe(percentiles=[.05,.1,.25,.5,.75,.9,.95])

## Data Cleaning

In [ ]:
list.isna().sum()

In [ ]:
# New Market Segment
list["NewMarketSegment"].value_counts()

* Seeing that Prime Market without Foreign Stock is most dominant, we will fill our missing values with it.

In [ ]:
list["NewMarketSegment"] = list["NewMarketSegment"].fillna("Prime Market")
list["NewMarketSegment"].isna().sum()

In [ ]:
# Trade Date
list["TradeDate"].value_counts()

In [ ]:
list["TradeDate"] = list["TradeDate"].fillna(list["TradeDate"].mean())
list["TradeDate"].isna().sum()

In [ ]:
# Close
list.Close.value_counts()

In [ ]:
list.Close = list.Close.fillna(list.Close.mean())
list.Close.isna().any()

In [ ]:
# Issued Shares
list.IssuedShares = list.IssuedShares.fillna(list.IssuedShares.mean())
list.IssuedShares.isna().any()

In [ ]:
# Market Capitalization
list.MarketCapitalization = list.MarketCapitalization.fillna(list.MarketCapitalization.mean())
list.MarketCapitalization.isna().any()

* Looking at `33SectorCode` and `17SectorCode` they seem to be represenetations of different sectors of the economy which have also been named so will just discard them.

In [ ]:
list.drop(["33SectorCode", "17SectorCode"], axis = 1, inplace = True)
list.head()

# Visualization

In [ ]:
plt.style.use("ggplot")

# We will plot a Pairplot of the numerical features
list_num = ["Close", "IssuedShares", "MarketCapitalization"]
sns.pairplot(list[list_num])

In [ ]:
corr = list[list_num].corr()
f,ag=plt.subplots(figsize=(16,16))
sns.heatmap(corr, cmap="spring")

In [ ]:
fig = plt.figure()
list[list_num].plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
plt.show()

In [ ]:
list[list_num].hist()

* Let us get an even keen look at every feature



# Close vs Date

In [ ]:
list.head()

In [ ]:
list.TradeDate.dtype

* Sometimes UNIX timestamps have to be converted to prevent them from changing to default

In [ ]:
list['TradeDate'] = pd.to_datetime(list['TradeDate'].astype("int64"), format = "%Y%m%d").dt.strftime('%Y-%m-%d') 
list.head()

In [ ]:
list.groupby("TradeDate")["Close"].mean().plot()

In [ ]:
list['EffectiveDate'] = pd.to_datetime(list['EffectiveDate'].astype("int64"), format = "%Y%m%d").dt.strftime('%Y-%m-%d') 
list.head()

# Issued Shares v. Date

In [ ]:
import seaborn as sns
plt.figure(figsize = (16,9)) 
list.groupby("TradeDate")["IssuedShares"].mean().plot()

# New Market Segment

In [ ]:
sns.countplot(list.NewMarketSegment)
plt.tick_params(axis='x', which='major', labelsize=15, rotation=90);

# Section/Products

In [ ]:
sns.countplot(list["Section/Products"])
plt.tick_params(axis="x", which = "major", rotation = 90);

In [ ]:
shares = list.groupby("EffectiveDate")["IssuedShares"].sum().to_frame().reset_index()
shares

In [ ]:
shares["EffectiveDate"] = pd.to_datetime(shares["EffectiveDate"])
shares["EffectiveDate"].head()

In [ ]:
import calendar

shares["year"] = shares.EffectiveDate.dt.year
shares

# Shares v. Years

In [ ]:
plt.plot(shares["year"], shares["IssuedShares"]);

In [ ]:
list.TradeDate.value_counts()

# Company v. Market Capitalization

In [ ]:
cap = list.groupby("MarketCapitalization")["Name"].value_counts().to_frame()
cap

# TODO
* Impute the missing values in `33 Sector` by finding the companies categories